In [1]:
import nltk
from nltk import word_tokenize
from nltk import ngrams
from nltk.stem import PorterStemmer as ps, WordNetLemmatizer as lemmatizer
from nltk.corpus import stopwords
import pandas as pd

In [3]:
data = pd.read_csv("./tripadvisor_hotel_reviews.csv")

In [4]:
data.info()

data["review_lowercase"] = data["Review"].str.lower()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  109 non-null    object
 1   Rating  109 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


,Review,Rating,review_lowercase
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...,2,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,3,nice rooms not 4* experience hotel monaco seat...
3,"unique, great stay, wonderful time hotel monac...",5,"unique, great stay, wonderful time hotel monac..."
4,"great stay great stay, went seahawk game aweso...",5,"great stay great stay, went seahawk game aweso..."


In [24]:
stopwords = stopwords.words('english')
stopwords.remove("not")

In [39]:
data['review_no_stopwords'] = data['review_lowercase'].str.split().apply(lambda words: ' '.join(
        [w for w in words if w not in stopwords]
    ))
data['review_no_stopwords']

0      nice hotel expensive parking got good deal sta...
1      ok nothing special charge diamond member hilto...
2      nice rooms not 4* experience hotel monaco seat...
3      unique, great stay, wonderful time hotel monac...
4      great stay great stay, went seahawk game aweso...
                             ...                        
104    fairmont hotel singapore splendid, stayed fair...
105    great hotel stayed hotel night route perth chr...
106    great hotel centre stayed 5 nights room overlo...
107    great location great value great hotel husband...
108    lovely hotel great location stayed vacation to...
Name: review_no_stopwords, Length: 109, dtype: object

In [49]:
data['review_no_stopwords_no_punct'] = data['review_no_stopwords'].str.replace(
    r'\*',
    'star',
    regex=True
)

data['review_no_stopwords_no_punct'] = data['review_no_stopwords_no_punct'].str.replace(
    r'[^\w\s]',
    '',
    regex=True
)

data['review_no_stopwords_no_punct']

0      nice hotel expensive parking got good deal sta...
1      ok nothing special charge diamond member hilto...
2      nice rooms not 4star experience hotel monaco s...
3      unique great stay wonderful time hotel monaco ...
4      great stay great stay went seahawk game awesom...
                             ...                        
104    fairmont hotel singapore splendid stayed fairm...
105    great hotel stayed hotel night route perth chr...
106    great hotel centre stayed 5 nights room overlo...
107    great location great value great hotel husband...
108    lovely hotel great location stayed vacation to...
Name: review_no_stopwords_no_punct, Length: 109, dtype: object

In [57]:
data['tokenize'] = data['review_no_stopwords_no_punct'].apply(word_tokenize)
data['tokenize']

0      [nice, hotel, expensive, parking, got, good, d...
1      [ok, nothing, special, charge, diamond, member...
2      [nice, rooms, not, 4star, experience, hotel, m...
3      [unique, great, stay, wonderful, time, hotel, ...
4      [great, stay, great, stay, went, seahawk, game...
                             ...                        
104    [fairmont, hotel, singapore, splendid, stayed,...
105    [great, hotel, stayed, hotel, night, route, pe...
106    [great, hotel, centre, stayed, 5, nights, room...
107    [great, location, great, value, great, hotel, ...
108    [lovely, hotel, great, location, stayed, vacat...
Name: tokenize, Length: 109, dtype: object

In [68]:
ps = PorterStemmer()
data['stemmed'] = data['tokenize'].apply(
    lambda tokens: [ps.stem(tok) for tok in tokens]
)
data['stemmed']

0      [nice, hotel, expens, park, got, good, deal, s...
1      [ok, noth, special, charg, diamond, member, hi...
2      [nice, room, not, 4star, experi, hotel, monaco...
3      [uniqu, great, stay, wonder, time, hotel, mona...
4      [great, stay, great, stay, went, seahawk, game...
                             ...                        
104    [fairmont, hotel, singapor, splendid, stay, fa...
105    [great, hotel, stay, hotel, night, rout, perth...
106    [great, hotel, centr, stay, 5, night, room, ov...
107    [great, locat, great, valu, great, hotel, husb...
108    [love, hotel, great, locat, stay, vacat, tour,...
Name: stemmed, Length: 109, dtype: object

In [72]:
lemmatizer = WordNetLemmatizer()

data['lemmatized'] = data['tokenize'].apply(
    lambda tokens: [lemmatizer.lemmatize(tok) for tok in tokens]
)

data['lemmatized']

0      [nice, hotel, expensive, parking, got, good, d...
1      [ok, nothing, special, charge, diamond, member...
2      [nice, room, not, 4star, experience, hotel, mo...
3      [unique, great, stay, wonderful, time, hotel, ...
4      [great, stay, great, stay, went, seahawk, game...
                             ...                        
104    [fairmont, hotel, singapore, splendid, stayed,...
105    [great, hotel, stayed, hotel, night, route, pe...
106    [great, hotel, centre, stayed, 5, night, room,...
107    [great, location, great, value, great, hotel, ...
108    [lovely, hotel, great, location, stayed, vacat...
Name: lemmatized, Length: 109, dtype: object

In [82]:
tokens_series = data['lemmatized'].explode()
tokens_series.value_counts()

lemmatized
hotel           292
room            275
great           126
not             122
stay             95
               ... 
175               1
smackagainst      1
2x                1
80                1
connected         1
Name: count, Length: 2589, dtype: int64

In [108]:
tokens_clean = sum(data['lemmatized'], [])

In [2]:
bigrams = (pd.Series(ngrams(tokens_series, 2)).value_counts())
bigrams

NameError: name 'tokens_series' is not defined